<a href="https://colab.research.google.com/github/Arc-blroth/IntelliTater/blob/main/IntelliTater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IntelliTater

IntelliTater is a 100% automated way to generate tater background splash screens for IntelliJ-family IDEs.

The algorithm used is currently taliored to the 2021.x style of splashes, but it shouldn't be too hard to edit the algorithm to handle other versions.

## Usage

1. Find the splash screen for your IDE inside `lib/resources.jar` from the root directory of your IDE installation. The filenames of the splash screen images vary between IDEs and Community/Ultimate editions. For IntelliJ IDEA Community Edition, the files are named `idea_community_logo.png` and `idea_community_logo@2x.png` for the splash screen and large splash screen respectively.

   The algorithm has mostly been tested on the double sized splash screens. To generate a normal sized splash screen, you can resize the 2x splash screen or tweak the `TEXT_DILATION_KERNEL` and `TEXT_BLUR_KERNEL` variables in the first cell.
2. Upload that splash screen to Colab (or copy it so that your Jupyter instance can find it), and rename it to `splash.png`.
3. If you want some background other than the default tater background, also upload a file named `background.png`.
4. Run the three cells below this README. (The other cells were random experiments to figure out better ways to separate the background of the splash from the foreground)
5. Download `out.png` from this notebook, and replace the splash file in `resources.jar` with that. (You might have to refresh the file list in Colab before `out.png` shows up)
6. Start your IDE and witness the power of Tiny Potato!

## Disclaimers

"Jetbrains", "IntelliJ", and "IntelliJ IDEA" are trademarks of [Jetbrains](www.jetbrains.com), which has not endorsed this taterfier.

The Tiny Potato is inspired from [Botania](https://botaniamod.net/).

The default background used in this notebook was created by [Arc-blroth](http://github.com/Arc-blroth) and is licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/legalcode). It is a derivative of the [Fabric Splash](https://github.com/FabricMC/community/blob/c217e169ccab96d4eed86606df1c6456879382d3/media/tater-wall/png/fabric-splash.png) by [gdude2002](https://github.com/gdude2002), licensed under the same license.

## Special Thanks

To [ramidzkh](https://github.com/ramidzkh), whose "who did this" post sparked the first tater splash.

To [Gudenau](https://github.com/gudenau), who asked me to make more of these.

To everyone else who downloaded and used the first tater splashes. Though it may have been deleted from Fabricord, #tiny-potato will live in our hearts forever!

In [431]:
!wget https://resources.jetbrains.com/storage/products/jetbrains/docs/jetbrains-logos-pack.zip -O jetbrains.zip
!unzip jetbrains.zip -d ./jetbrains
!mv ./jetbrains/jetbrains-logos/blackandwhite/jetbrains-blackandwhite.png ./
!rm -rf ./jetbrains/
!rm jetbrains.zip

from IPython.display import clear_output
clear_output()

# Logo text adjustment factors
# ----------------------------
# These are used to compensate for the fact that the
# text in the original splash is rendered subpixel, but
# OpenCV can only reliably detect to pixel accuracy.
#
# The mask that controls what parts of the image is
# interpreted as "logo" text is expanded by the dilation
# kernal and then filter blurred by the blur kernel.
TEXT_DILATION_KERNEL = (1, 1)
TEXT_BLUR_KERNEL = (2, 2)
TEXT_BLUR_CUTOFF = 2

In [432]:
from IPython.display import clear_output
from os import path
import numpy as np
import matplotlib.pyplot as plt
import cv2

assert path.exists("splash.png"), "No `splash.png` file found!"

# Download background.png if it doesn't already exist
if not path.exists("background.png"):
    print("No `background.png` file found, downloading default tater background...")
    !wget https://raw.githubusercontent.com/Arc-blroth/IntelliTater/main/intellitater_wall.png -O background.png -q

# Read images
img = cv2.cvtColor(cv2.imread("splash.png"), cv2.COLOR_BGR2RGB)
background = cv2.cvtColor(cv2.imread("background.png"), cv2.COLOR_BGR2RGB)
background = background[0 : img.shape[0], 0 : img.shape[1]]

# Step 0: Crop logo
logo = cv2.cvtColor(
    cv2.imread("jetbrains-blackandwhite.png", cv2.IMREAD_UNCHANGED), cv2.COLOR_BGRA2RGBA
)
logo[np.all(logo == (0, 0, 0, 0), axis=-1)] = (255, 255, 255, 255)
logo_thresh = cv2.cvtColor(logo, cv2.COLOR_RGBA2GRAY)
_, logo_thresh = cv2.threshold(logo_thresh, 127, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(logo_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
x, y, w, h = cv2.boundingRect(contours[0])
logo = logo[y : y + h, x : x + w]
logo = cv2.cvtColor(logo, cv2.COLOR_RGBA2RGB)

In [ ]:
# Step 1: Figure out where text is
text_mask = cv2.inRange(img, np.array([200, 200, 200]), np.array([255, 255, 255]))
text_mask = cv2.dilate(text_mask, np.ones(TEXT_DILATION_KERNEL, np.uint8), iterations=1)
text_mask = cv2.filter2D(text_mask, -1, np.ones(TEXT_BLUR_KERNEL, np.float32) / TEXT_BLUR_CUTOFF)

# Step 2: Figure out where logo is
logo_mask = cv2.inRange(img, np.array([0, 0, 0]), np.array([10, 10, 10]))
contours, _ = cv2.findContours(logo_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
  x, y, w, h = cv2.boundingRect(contour)
  cv2.rectangle(logo_mask, (x, y), (x + w - 1, y + h - 1), (255, 255, 255), -1)

# Step 3: Feather and combine
# See https://stackoverflow.com/a/55970627
float_mask = cv2.cvtColor(text_mask, cv2.COLOR_GRAY2BGR).astype(np.float32) / 255
float_img = img.astype(np.float32) / 255
float_background = background.astype(np.float32) / 255
# For better masking alpha is multiplied by how close the color is to pure white
actual_mask = np.zeros(float_img.shape)
actual_mask[:, :, 0] = np.average(float_img * float_mask, axis=2)
actual_mask[:, :, 1] = actual_mask[:, :, 0]
actual_mask[:, :, 2] = actual_mask[:, :, 0]
out = float_background * (1 - actual_mask) + np.ones(float_img.shape) * actual_mask
out = (out * 255).astype(np.uint8)
cv2.copyTo(img, logo_mask, out)

cv2.imwrite("out.png", cv2.cvtColor(out, cv2.COLOR_RGB2BGR))
# cv2.imwrite("logo_found_img.png", cv2.cvtColor(logo_found_img, cv2.COLOR_RGB2BGR))
print("Taterification complete!")

if "google.colab" in str(get_ipython()):
    from ipywidgets import widgets
    from IPython.display import display
    from IPython.core.display import HTML
    from google.colab import files

    def download_out(e):
        files.download("out.png")

    download_button = widgets.Button(description="Download `out.png`")
    download_button.on_click(download_out)
    download_button.margin = "20"
    display(widgets.VBox([widgets.HBox([download_button])]))

plt.imshow(out)
plt.show()

-- Other Experiments --

In [ ]:
# Find logo in the image using template matching
# based on https://www.pyimagesearch.com/2015/01/26/multi-scale-template-matching-using-python-opencv/
# but scales the template rather than the image and uses binary search instead of brute force steps
def find_scaled_template(image, template, lower_size, upper_size, steps):
    (tH, tW) = template.shape[:2]
    step = (upper_size - lower_size) / 2
    scale = lower_size + step / 2
    step /= 2
    found = None

    for i in range(steps):
        lower_scale = scale - step / 2
        upper_scale = scale + step / 2
        lower_resized = cv2.resize(
            template,
            (
                int(template.shape[1] * lower_scale),
                int(template.shape[0] * lower_scale),
            ),
        )
        upper_resized = cv2.resize(
            template,
            (
                int(template.shape[1] * upper_scale),
                int(template.shape[0] * upper_scale),
            ),
        )

        lower_result = cv2.matchTemplate(image, lower_resized, cv2.TM_CCORR_NORMED)
        upper_result = cv2.matchTemplate(image, upper_resized, cv2.TM_CCORR_NORMED)

        _, lower_max_val, _, lower_max_loc = cv2.minMaxLoc(lower_result)
        _, upper_max_val, _, upper_max_loc = cv2.minMaxLoc(upper_result)
        if lower_max_val > upper_max_val:
            found = (lower_max_val, lower_max_loc, lower_scale)

        else:
            found = (upper_max_val, upper_max_loc, upper_scale)

        scale = found[2]
        step /= 2

    _, maxLoc, r = found
    return (int(maxLoc[0]), int(maxLoc[1]), int(tW * scale), int(tH * scale))


logo_found_img = img.copy()
x, y, w, h = find_scaled_template(logo_found_img, logo, 0.1, 0.2, 10)
logo_found_img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
plt.imshow(logo_found_img)
plt.show()

In [ ]:
!apt install tesseract-ocr
!pip install pytesseract

In [ ]:
# OCR
import pytesseract

# image is inverted because tesseract expects black-on-white
inv_img = cv2.bitwise_and(img, img, mask=mask)
inv_img = 255 - inv_img

data = pytesseract.image_to_data(inv_img, output_type=pytesseract.Output.DICT)
for i in range(len(data["level"])):
    if data["text"][i]:
        x, y, w, h = (
            data["left"][i],
            data["top"][i],
            data["width"][i],
            data["height"][i],
        )
        cv2.rectangle(inv_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

plt.imshow(inv_img)
plt.show()